# Tunisian Horses - Only stills
* do not use the cropped images, because the cropped images are most probably extracted from the stills

## Create metadata

In [2]:
import shutil
import os
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

In [3]:

# 2. Prepare data for model training:
# Base path to the database
base_dir = '../data/THoDBRL2015'

metadata = []

# Consolidate images into a training directory
output_train_dir = os.path.join(base_dir, 'training_data')

os.makedirs(output_train_dir, exist_ok=True)

# Iterate through each Part folder
for part in ['Part1', 'Part2', 'Part3', 'Part4', 'Part5']:
    videos_dir = os.path.join(base_dir, part, 'videos')

    for horse_id_folder in os.listdir(videos_dir):
        horse_path = os.path.join(videos_dir, horse_id_folder)

        if os.path.isdir(horse_path):  # Ensure it's a directory
            # Find all stills folders (e.g., images, images1, images2, etc.)
            for stills_folder in os.listdir(horse_path):
                stills_path = os.path.join(horse_path, stills_folder)

                if os.path.isdir(stills_path) and stills_folder.startswith('images'):  # Check for folders named 'images*'
                    target_dir = os.path.join(output_train_dir, f'horse_{horse_id_folder}')

                    os.makedirs(target_dir, exist_ok=True)
                    
                    # Copy all image files from the stills folder to the target directory
                    for img_file in os.listdir(stills_path):
                        img_path = os.path.join(stills_path, img_file)

                        if img_file.endswith(('.jpg', '.jpeg', '.png')):  # Ensure it's an image file
                            metadata.append({
                                'horse_id': horse_id_folder,
                                'image_path': img_path,
                                'width': Image.open(img_path).size[0],
                                'height': Image.open(img_path).size[1],
                            })
                        
                            # shutil.copy(img_path, target_dir)

                    # print(f"Copied images from {stills_path} to {target_dir}")

# Create DataFrame with specified dtypes
metadata_df = pd.DataFrame(metadata, dtype='object').astype({
    'horse_id': 'int64',          # Assuming it's an integer
    'image_path': 'string',
    'width': 'int64',
    'height': 'int64',
})




## Descriptive Analysis

In [4]:
print(f"Total images: {len(metadata_df)}")
print(f"Total horses: {metadata_df['horse_id'].nunique()}")
print(f"Width: {metadata_df['width'].unique()}")
print(f"Height: {metadata_df['height'].unique()}")


Total images: 60971
Total horses: 47
Width: [640]
Height: [480]


In [5]:
print(metadata_df[:5])

   horse_id                                         image_path  width  height
0         9  ../data/THoDBRL2015/Part1/videos/9/images1/img...    640     480
1         9  ../data/THoDBRL2015/Part1/videos/9/images1/img...    640     480
2         9  ../data/THoDBRL2015/Part1/videos/9/images1/img...    640     480
3         9  ../data/THoDBRL2015/Part1/videos/9/images1/img...    640     480
4         9  ../data/THoDBRL2015/Part1/videos/9/images1/img...    640     480


In [6]:
grouped_horse = metadata_df.groupby(['horse_id'], observed=True)

grouped_horse.describe()

width                                                 height  \
           count   mean  std    min    25%    50%    75%    max   count   
horse_id                                                                  
1         1415.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1415.0   
2         1029.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1029.0   
3         1282.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1282.0   
4         1173.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1173.0   
5          548.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   548.0   
6          729.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   729.0   
7         1226.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1226.0   
8         1262.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1262.0   
9          585.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   585.0   
10        1115.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1115.0   
11        1248.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1248.0   
12         679.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   679.0   
13        1396.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1396.0   
14        2257.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  2257.0   
15        1597.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1597.0   
16        1292.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1292.0   
17         753.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   753.0   
18         598.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   598.0   
19         786.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   786.0   
20         808.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   808.0   
21        1168.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1168.0   
22         752.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   752.0   
23        1025.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1025.0   
24        1591.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1591.0   
25        3475.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  3475.0   
26         825.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   825.0   
27         566.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   566.0   
28         872.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   872.0   
29        1136.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1136.0   
30        1276.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1276.0   
31         931.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   931.0   
32         670.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   670.0   
33         661.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   661.0   
34        1247.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1247.0   
35         698.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   698.0   
36        1746.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1746.0   
37        1351.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1351.0   
38        2579.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  2579.0   
39        1356.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1356.0   
40        1591.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1591.0   
41        1883.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1883.0   
42        3762.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  3762.0   
43         991.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   991.0   
44        2705.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  2705.0   
45        1831.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1831.0   
46        1592.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0  1592.0   
47         913.0  640.0  0.0  640.0  640.0  640.0  640.0  640.0   913.0   

                                                         
           mean  std    min    25%    50%    75%    max  
horse_id                                                 
1         480.0  0.0  480.0  480.0  480.0  480.0  480.0  
2         480.0  0.0  480.0 

In [7]:
# Images per horse id
grouped_horse.size().agg(['min', 'max', 'mean'])

min      548.000000
max     3762.000000
mean    1297.255319
dtype: float64

In [8]:
df = pd.DataFrame(metadata_df)

## Prepare dataset

In [9]:
# For splitting data
# Load data
# Prepare dataset
def load_images_and_labels(metadata_df):
    images = []
    labels = []
    
    for _, row in metadata_df.iterrows():
        if os.path.exists(row['image_path']):
            img = Image.open(row['image_path']).resize((128, 128))  # Scale to same size
            images.append(np.array(img))
            labels.append(row['horse_id'])  # use horse_id for the label
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels
    


In [10]:
# Load data
images, labels = load_images_and_labels(metadata_df)


### Split in train and test

In [11]:

# Split in train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

In [12]:
# Normalise pixelvalues
# X_train = X_train / 255.0
# X_test = X_test / 255.0

# Transfer labels to category data
# num_classes = len(metadata_df['horse_id'].unique())
num_classes = len(np.unique(labels))


y_train = y_train - 1  # If needed then custumize labels (sometimes gives error)
y_test = y_test - 1  # The same for test

# print(y_train.min())  # Min label value
# print(y_train.max())  # Max label value

# Convert labels to categorical data
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [13]:
# Normalise pixel values?

## Create CNN

In [14]:
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model():
    model = Sequential()

    # Add layers step-by-step
    model.add(keras.Input(shape=(128, 128, 3)))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # To prevent overfitting

    model.add(Dense(num_classes, activation='softmax'))  # Output layer
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


# Create a model
model = create_model()

# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 47)             │         6,063 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,310,703 (12.63 MB)

 Trainable params: 3,310,703 (12.63 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
y_train.shape
y_train[:5]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

In [16]:
# [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,0 ,0] = one hot encoding

In [19]:
history = model.fit(
    X_train, y_train,
    epochs=1,
    batch_size=128,
    validation_data=(X_test, y_test)
)


382/382 ━━━━━━━━━━━━━━━━━━━━ 376s 983ms/step - accuracy: 0.9608 - loss: 0.1214 - val_accuracy: 0.9970 - val_loss: 0.0086


Findings with epochs=10 and batch_size=32
- expected time (335 sec * 10 epochs) ~~ 1 hour
- Accuracy in 1st epoch: 0.54, and loss 3.92
- Accuracy in 2nd epoch: on the start already accuracy of 0.91
- In 2nd epoch, no big changes.

Findings with epochs=1 and batch_size=64
- expected time ~6 min (so same time per epoch)
- accuracy is >0.95.

Explanation:
- 

## Save model

In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save('../data/saved_models/my_model_epoch_1_batch_size_128.keras')

## Confusion matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

class_names = [str(i) for i in range(metadata_df['horse_id'].nunique())]  

# predict the labels for testset
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)

# Plot The confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Class')
plt.ylabel('Real Class')
plt.title('Confusion Matrix')
plt.show()